In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

Inês deseja abrir uma loja de produtos diversos. Ao ir para o fornecedor Fictício, ela recebeu um menu com diversas produtos e algumas informações relevantes sobre eles. Inês irá primeiro escolher os produtos para depois determinar a quantidade deles a ser comprada

O menu fornecido pela fazenda encontra-se no arquivo "produtos_a.csv"

O arquivo contém as seguintes colunas:



*   Produto: indica o produto a ser comprado
*   Custo_compra: o valor da unidade para comprar do fornecedor Fictício em reais
*   Lucro: o lucro esperado para Inês por unidade do produto em reais
*   Valorização: Uma escala de zero a dez, com precisão de duas casas decimais, que indica a expectativa do lucro aumentar nos próximos meses
*   Demanda: Quantidade do produto comprada no mês anterior
*   Departamento: Qual o departamento do produto sugerido pelo fornecedor Fictício

O arquivo já está pronto para uso, não sendo necessário eliminar linhas ou colunas por problemas na coleta e compilação dos dados




Vamos ler o arquivo de dados

In [17]:
#vamos definir uma função que sempre leia o arquivo e retorne o dataset, assim, em qualquer célula podemos invocar o dataset sem qualquer alteração

df = pd.read_csv("produtos_a.csv")
df

,produto,custo_compra,lucro,valorizacao,demanda,departamento
0,Agulha,10.94,50.28,1.89,1636.6,cozinha
1,Alfinete,32.87,176.73,4.04,2969.9,cozinha
2,Anel,26.54,52.55,4.79,2130.0,eletrodomestico
3,Aro,24.95,102.11,4.31,4343.4,cozinha
4,Anzol,17.88,70.75,4.55,2513.9,sala
...,...,...,...,...,...,...
159,Xicara,23.76,103.14,2.39,4498.1,eletrodomestico
160,Xadrez,22.69,166.43,6.36,1493.3,decoracao
161,Xale,28.65,55.11,5.53,3273.3,eletrodomestico
162,Xarope,53.85,18.19,6.26,1329.9,cozinha


## Item 1

Auxilie Inês a escolher os melhores produtos elaborando um modelo de otimização que maximize a demanda - assim, espera-se que haja garantia de venda

### a) Defina um modelo para ajudar na tomada de decisão de Inês

Conjuntos\
$F →$ Conjunto de produtos

Parâmetros:\
$I_f →$ Demanda para um produto $f \in F$

Variáveis de decisão:\
$x_f →$ Binária. Indica se o produto $f \in F$ deve ser nos melhores

Função Objetivo:\
max $\sum_{f \in F} I_fx_f$

Restrições:\
$\sum_{f \in F} x_f = 8$

### b) Resolva o modelo com um resolvedor

In [18]:
m = gp.Model()

F = list(df['produto'])

I_f = list(df['demanda'])

x = []
for i in range(len(F)):
  x.append(m.addVar(vtype=GRB.BINARY, name=F[i]))

m.setObjective(gp.quicksum(x[i]*I_f[i] for i in range(len(F))), GRB.MAXIMIZE)

m.addConstr(gp.quicksum(x[i] for i in range(len(F))) == 8)

m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9700KF CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 164 columns and 164 nonzeros
Model fingerprint: 0xcd3bcb89
Variable types: 0 continuous, 164 integer (164 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+00, 8e+00]
Found heuristic solution: objective 22631.400000
Presolve removed 1 rows and 164 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 38133.8 22631.4 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.813380000000e+04, best bound 3.813380000000e+04, gap 0.0000%


In [19]:
produtos = []
for i in range(len(F)):
  if x[i].x == 1:
    produtos.append([F[i], I_f[i]])

produtos_df = pd.DataFrame(produtos,columns=['Produto', 'Demanda'])
produtos_df

,Produto,Demanda
0,Carimbo,4766.2
1,Facão,4697.3
2,Insecticida,5093.0
3,Lampada,4650.6
4,Quebra Cabeça,5041.9
5,Tesoura,4793.7
6,Umidificador de Ar,4509.3
7,Vela,4581.8


In [20]:
print("Valor ótimo: ", m.objVal)
print("Solução ótima: ", m.Status == GRB.OPTIMAL)

Valor ótimo:  38133.8
Solução ótima:  True


### c) Discorra sobre a solução obtida

## Item 2

Considere que a loja terá somente trinta e cinco espécies de árvores frutíferas

### d) Altere o modelo para representar esta nova situação

### e) Resolva o problema e discorra sobre a solução obtida

## Item 3

Devido a questões logística, o fornecedor possui somente produtos de demanda no intervalo [1100, 3800]

### f) Altere o modelo para atender esta nova situação

In [24]:
new_df = df[(df['demanda'] >= 1100) & (df['demanda'] <= 3300)]
new_df

,produto,custo_compra,lucro,valorizacao,demanda,departamento
0,Agulha,10.94,50.28,1.89,1636.6,cozinha
1,Alfinete,32.87,176.73,4.04,2969.9,cozinha
2,Anel,26.54,52.55,4.79,2130.0,eletrodomestico
4,Anzol,17.88,70.75,4.55,2513.9,sala
5,Apontador,54.79,55.65,2.32,2180.0,sala
...,...,...,...,...,...,...
156,Vidro,72.32,37.85,3.98,2943.9,eletrodomestico
157,Viseira,53.44,178.64,6.26,1795.8,eletrodomestico
160,Xadrez,22.69,166.43,6.36,1493.3,decoracao
161,Xale,28.65,55.11,5.53,3273.3,eletrodomestico


In [25]:
n = gp.Model()

F = list(new_df['produto'])

I_f = list(new_df['demanda'])

x = []
for i in range(len(F)):
  x.append(n.addVar(vtype=GRB.BINARY, name=F[i]))

n.setObjective(gp.quicksum(x[i]*I_f[i] for i in range(len(F))), GRB.MAXIMIZE)

n.addConstr(gp.quicksum(x[i] for i in range(len(F))) == 8)

n.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9700KF CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 100 columns and 100 nonzeros
Model fingerprint: 0xfaacc34c
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+03, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+00, 8e+00]
Found heuristic solution: objective 19414.200000
Presolve removed 1 rows and 100 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 26016 19414.2 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.601600000000e+04, best bound 2.601600000000e+04, gap 0.0000%


In [26]:
produtos = []
for i in range(len(F)):
  if x[i].x == 1:
    produtos.append([F[i], I_f[i]])

produtos_df = pd.DataFrame(produtos,columns=['Produto', 'Demanda'])
produtos_df

,Produto,Demanda
0,Cola,3222.7
1,Espada,3277.5
2,Folha,3280.5
3,Garrafa,3231.2
4,Harpa,3242.1
5,Luvas,3239.7
6,Unha Postiça,3249.0
7,Xale,3273.3


In [27]:
print("Valor ótimo: ", m.objVal)
print("Solução ótima: ", m.Status == GRB.OPTIMAL)

Valor ótimo:  38133.8
Solução ótima:  True


### g) Resolva o problema e discorra sobre a solução obtida

## Item 4

Devido a seu interesse em expandir seus lucros, Inês decidiu escolher produtos de acordo com a sua possibilidade de valorização, ao invés da demanda

### h) Altere o modelo para atender esta nova situação

### i) Resolva o problema e discorra sobre a solução obtida

## Item 5

Haja vista que a demanda não é o único fator determinante para o lucro, Inês decidiu que a demanda total dos produtos do jardim não deve ser superior a 80000

### j) Altere o modelo para atender esta nova situação

### k) Resolva o problema e discorra sobre a solução obtida

## Item 6

A partir do modelo elaborado, vamos explorar alguns parâmetros do resolvedor

### l) Pesquise e explique o papel dos parâmetros "PoolSearchMode" e "PoolSolutions"

### m) Indique se há mais de uma solução ótima para o problema

## Item 7

Façamos um último acréscimo ao modelo: Inês deseja garantir um lucro mínimo por meio da venda de produtos. Para isso, ela deseja que cada um dos produtos escolhidos tenham um lucro de, pelo menos, cento e cinquenta reais

### n) Altere o modelo para atender esta nova situação

### o) Resolva o problema e discorra sobre a solução obtida

## Item 8

A partir dos problemas desenvolvidos nesta questão responda:

### p) Como o acréscimo de restrições interfere na solução ótima do problema?